### IS - Brand position
Finding Product position on Social Network (PPSN)

In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
import itertools
import math
import string
import json
import pymongo
from itertools import groupby

In [ ]:
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=pd.core.common.SettingWithCopyWarning)

In [ ]:
import time
from datetime import datetime
today = datetime.now()

In [ ]:
#!pip install pythainlp
import pythainlp
from pythainlp import Tokenizer
from pythainlp.util import normalize
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_words
from pythainlp import sent_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.util import dict_trie
from pythainlp.tag import pos_tag

In [ ]:
# https://thainlp.org/pythainlp/docs/2.0/api/tag.html
from pythainlp.spell import correct
from pythainlp.tag import pos_tag

#https://thainlp.org/pythainlp/docs/2.0/api/transliterate.html
from pythainlp.transliterate import romanize

In [ ]:
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import jaccard_score
import scipy.stats
from scipy.spatial.distance import cosine
from mlxtend.evaluate import lift_score
from gensim.models import TfidfModel

In [ ]:
# Files definition
root_input_path = 'Datasource'
root_output_path = 'output'

#comment_file = []
#comment_file.append(root_input_path+"/comment_meiji.csv")
#comment_file.append(root_input_path+"/comment_DDCF.csv")

#comment_file = []
#comment_file.append("comment_meiji.csv")
#comment_file.append("comment_DDCF.csv")

comment_nlp_file = root_output_path+"/comment_nlpToken.csv"
comment_cooc_freq_file = root_output_path+"/comment_cooc_freq.xlsx"
comment_cooc_lift_file = root_output_path+"/comment_cooc_Lift.xlsx"
comment_cooc_jacc_file = root_output_path+"/comment_cooc_jacc.xlsx"
comment_cooc_corr_file = root_output_path+"/comment_cooc_corr.xlsx"
comment_cooc_cosine_file = root_output_path+"/comment_cooc_cosine.xlsx"

#comment_nlp_file = "comment_nlpToken.csv"
#comment_cooc_nlp_file = "comment_cooc.xlsx"

In [ ]:
# Connect MongoDB
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["NIDA_PPSN_PRD"]
col_thread = mydb["NIDA_PPSN_THREAD"]
col_comment = mydb["NIDA_PPSN_COMMENT"]
col_scrape = mydb["NIDA_PPSN_SCRAPE"]

### 1.Clean data & NLP Tokenized

In [ ]:
def checkStopwords(w):
    returnList = []
    stopwords = set(thai_stopwords())
    stopwords.remove('ไม่')
    stopwords.remove('ได้')
    stopwords.remove('สูง')
    for i in w:  # comment
        if i not in stopwords:
            returnList.append(i)
    return returnList

In [ ]:
def normThai(w):
    returnList = []
    for i in w:
        returnList.append(normalize(i).lower())
    return returnList

In [ ]:
list_punc = []
for c in string.punctuation:
    list_punc.append(c)

In [ ]:
cursor_corpus = col_scrape.find()
df_Corpus = pd.DataFrame(cursor_corpus)

In [ ]:
df_Corpus.head()

In [ ]:
print('All comment:',df_Corpus.shape[0])
print('All URLs ก่อนตัด:',len(df_Corpus.URLs.unique()))

In [ ]:
missing = df_Corpus.isnull().sum()
print(missing)

In [ ]:
df_Corpus.tail()

In [ ]:
df_Corpus.drop('_id',axis=1,inplace=True)

In [ ]:
seqNum = list(range(1,df_Corpus.shape[0]+1))
df_Corpus.insert(0,'commentId',seqNum)
df_Corpus.columns = ['commentId','URLs','headline','text','article_date','Retrived_date']

In [ ]:
df_Corpus.tail()

In [ ]:
# Drop NA & banned comments
df_Corpus = df_Corpus.dropna()
df_Corpus = df_Corpus[df_Corpus["text"].str.find('ความคิดเห็นนี้ถูกลบ')!= 0]
df_Corpus = df_Corpus[df_Corpus["text"].str.find('แก้ไขข้อความเมื่อ')!= 0]
df_Corpus = df_Corpus.set_index('commentId')

In [ ]:
missing = df_Corpus.isnull().sum()
print(missing)

In [ ]:
print('จำนวน comment หลังตัด:',df_Corpus.shape[0])
print('จำนวนกระทู้หลังตัด:',len(df_Corpus.URLs.unique()))

In [ ]:
df_Corpus.insert(4,'token_text',None)
df_Corpus.tail()

เช็ควันเวลา

In [ ]:
print('กระทู้เก่าที่สุด :',min(df_Corpus.article_date))
print('กระทู้ใหม่ที่สุด :',max(df_Corpus.article_date))

เก็บเฉพาะหัวกระทู้

In [ ]:
df_thread = df_Corpus.copy()
df_thread = df_thread[['URLs','headline']]
df_thread.drop_duplicates(subset ="URLs", keep = 'first', inplace = True)
df_thread.insert(2,'token_headline',None)
df_thread.insert(3,'mention_product',None)
df_thread.insert(4,'mention_brand',None)
df_thread.tail()

## 2.ตัดคำและเก็บลง MongoDB (ใช้ insight จาก jupyter งานก่อน)

https://www.thainlp.org/pythainlp/tutorials/notebooks/pythainlp-get-started.html#Thai-Characters

In [ ]:
garbage_char = ['🥺','\r','\n','\r\n','\xa0','www','application','ทางกฎหมาย','html'
                ,'http','https','url','com','pantip','topic','_','–','พันทิป','^^'
                ,'นะค่ะ','นะคะ','น่ะค่ะ','ฯ','ๆ','แฮ่','เย่ๆๆ','ว๊าว','ฮิ้วววววววววว','Number','กก'
                ,'เมนู','Menu','Net','net','สาขา','บาท','ราคา','ฯ','ๆ','กก','อันนี้','😆', '🤣','😢'
                ,'😏','😂','😿','ววว','xx','อิอิ','แย้ววว','นะคะ','นะค่ะ','มกราคม', 'กุมภาพันธ์'
                ,'มีนาคม', 'เมษายน','พฤษภาคม', 'มิถุนายน', 'กรกฎาคม', 'สิงหาคม','กันยายน', 'ตุลาคม'
                ,'พฤศจิกายน', 'ธันวาคม']
garbage_char  = garbage_char + list_punc

In [ ]:
print('จำนวน comment หลังตัด:',df_Corpus.shape[0])
print('จำนวนกระทู้หลังตัด:',len(df_Corpus.URLs.unique()))

In [ ]:
custom_list = ['ดัชมิลล์','ดัชมิล','dutch mill','dutchmilk','ดัชชี่','เมจิ','ซีพีเมจิ','ซีพี-เมจิ','cp-meiji','betagen','บีทาเก้น','บีทาเกน'
               ,'โฟร์โมสต์','โฟร์โมส','โฟโมสต์','โฟโมต','โฟรโมสต์','โชคชัย','แดรี่โฮม','dairy home','เดรี่โฮม','อืมม มิลค์'
               ,'Umm! Milk','เอ็มมิลค์','m milk','mmilk','เอ็มมิ้ลค์','เอ็มมิลล์','แมคโนเลีย','นมฟาร์มโชคชัย','นมไทย-เดนมาร์ค','ไทยเดนมาร์ค','วัวแดง','หนองโพ','คาเนชั่น'
               ,'สตอเบอร์รี่','สตรอว์เบอร์รี','รสสตรอเบอรี่','สตรอเบอรี่','สตรอวเบอรี่','สตอเบอรี่','สตรอเบอรี่','รสกาแฟ'
               ,'ชอคโกแล็ต','ช็อกโกแลต','ช็อคโกแลต','ช๊อกโกแลต','ช็อคโกแล็ต','ช็อกโกเลต','มิ้นท์ชอค','มิ้นต์ช้อก','รสกล้วย'
               ,'รสกาแฟ','รสหวาน','รสจืด','ไขมันต่ำ','ไขมัน 0%','low fat','0% fat','ขาดมันเนย','พร่องมันเนย','ไฮแคลเซียม'
               ,'high protein','Hi Protein','HIgh Protein','Whey Formula','วานิลลา'
               ,'ไฮโปรตีน','เวย์โปรตีน','whey','เวย์','อัลมอนด์','ซีเล็ค','ซีเล็คท์','กล้วยหอม','ไวท์คอฟฟี่','คาปูชิโน่','อเมริกาโน่'
               ,'grass fed','free lactose','lactose free','พาสเจอไรซ์','พาสเจอร์ไรซ์','พาสเจอร์ไรส์','พาซเจอไรซ์','พาสเจอร์ไลท์'
               ,'เมจิโกลด์ แม็กซ์','เมจิโกลด์','เมจิโกล์ดแม็กซ์','เมจิ บัลแกเรีย','เมจิบัลแกเรีย','นมเปรี้ยว','รสธรรมชาติ','รสกลมกล่อม'
               ,'โยเกิต','โยเกิรต','โยเกิร์ตพร้อมดื่ม','น้ำตาลมะพร้าว','นมฮอกไกโด','เบดไทม์','ฟรีแลคโตส','แลคโตสฟรี','ริชเอสเพสโซ่'
               ,'bedtime','bed time','เบดไทม์','ดาร์คช็อกโกแลต','ดาร์กช็อก','ดาร์กช็อกโกแลต'
               ,'คาราเมล','ช็อกโกมอลต์','เมล่อน','ชาเขียวมัจฉะ','ฝาน้ำเงิน','ผลิตภันท์นม'
               ,'7-Eleven','เซเว่น อีเลฟเว่น','เซเว่นอีเลฟเว่น','เซเว่น','เซเวน','7/11','7-11'
               ,'ท็อปส์','ทอปส์','ท้อปส์','ท๊อปส์','แมคโคร','แม็คโคร','โลตัส','บิ๊กซี','bigc','Golden place','Big C'
               ,'ขายไม่ดี','แพคคู่','ค่าจัดส่ง','shelf life','พนักงานขายนม'               
               ,'ซื้อประจำ','ซื้อไม่ได้','ของแถม','ราคาสูง','น้ำนมโค'
               ,'แพ้นม','แพ้นมวัว','เล่นเวท','นำ้หนัก','คุณแม่มือใหม่','นมอุ่น','ชานม','กินนม','ดื่มนม','ท้องเสีย','ขี้แตก'
               ,'คีเฟอร์นม','พันทิป','ร้านนม','ซื้อไม่ได้','ลูกสุนัข','ลูกหมา','คายทิ้ง','เจมส์ จิ','เจมส์จิ','สตอรี่'
              ]
words = set(thai_words()).union(set(custom_list))
trie = dict_trie(dict_source=words)

In [ ]:
def tokenize_custom(x, cusDict):
    custom_punc = '!"#$&\'()*+,.:;<=>?@[\\]^_`{|}~'
    wtkn = x.translate(str.maketrans('', '', custom_punc)).strip()
    wtkn = word_tokenize(wtkn, engine="newmm", custom_dict=cusDict, keep_whitespace=False)
    wtkn = checkStopwords(wtkn)
    wtkn = [w for w in wtkn if len(w) >= 2]
    wtkn = [y for y in wtkn if y not in garbage_char]
    wtkn = [s for s in wtkn if not s.isdigit()]
    wtkn = normThai(wtkn)
    return wtkn

In [ ]:
def create_InsertMany_Thread(df):
    listofdict = []
    for c,lx in enumerate(df.URLs,0):
        info = {
            "URLs": lx,
            "headline": df.headline[c],
            "token_headline":df.token_headline[c],
            "t_mention_dairy":df.t_mention_dairy[c],
            "t_mention_product": df.t_mention_product[c],
            "t_mention_brand": df.t_mention_brand[c]
        }
        listofdict.append(info)
    return listofdict

In [ ]:
def create_InsertMany_comments(df):
    listofdict = []
    for c,lx in enumerate(df.URLs,0):
        info = {
            "commentId":int(df.commentId[c]),
            "URLs": lx,
            "headline":df.headline[c],
            "text": df.text[c],
            "Retrived_date": df.Retrived_date[c],
            "token_text":df.token_text[c],
            "cmt_mention_dairy": df.cmt_mention_dairy[c],
            "cmt_mention_product": df.cmt_mention_product[c],
            "cmt_mention_brand": df.cmt_mention_brand[c]
        }
        listofdict.append(info)
    return listofdict

In [ ]:
def generalize_keyword(wtkn):  
    ret_wtkn = wtkn
    for i,each in enumerate(ret_wtkn,0):
        if each in ['ดัชมิลล์','ดัชมิล','dutch mill','dutchmill','dutchmilk','ดัชชี่']:
            ret_wtkn[i] = 'ดัชมิลล์'
        elif each  in ['เมจิ','ซีพีเมจิ','ซีพี-เมจิ','cp-meiji','meiji','meji','miji']:
            ret_wtkn[i] = 'เมจิ'
        elif each  in ['โฟร์โมสต์','โฟร์โมส','โฟโมสต์','โฟรโมสต์','โฟโมต','foremost']:
            ret_wtkn[i] = 'โฟร์โมสต์'
        elif each  in ['แดรี่โฮม','dairy home','เดรี่โฮม']:
            ret_wtkn[i] = 'แดรี่โฮม'
        elif each  in ['โชคชัย','อืมม มิลค์','umm milk','นมฟาร์มโชคชัย']:
            ret_wtkn[i] = 'โชคชัย'
        elif each  in ['เอ็มมิลค์','mmilk','เอ็มมิ้ลค์','เอ็มมิลล์','m milk']:
            ret_wtkn[i] = 'เอ็มมิลค์'
        elif each  in ['นมไทย-เดนมาร์ค','ไทยเดนมาร์ค','วัวแดง']:
            ret_wtkn[i] = 'ไทยเดนมาร์ค'
        elif each  in ['สตอเบอร์รี่','สตรอว์เบอร์รี','รสสตรอเบอรี่','สตอเบอรี่','สตรอเบอรี่','สตรอวเบอรี่']:
            ret_wtkn[i] = 'สตรอว์เบอร์รี'
        elif each  in ['ชอคโกแล็ต','ช็อกโกแลต','ช็อคโกแลต','ช๊อกโกแลต','ช็อคโกแล็ต','ช็อกโกเลต','ช็อค']:
            ret_wtkn[i] = 'ช็อกโกแลต'
        elif each  in ['มิ้นท์ชอค','มิ้นต์ช้อก','ช็อคโกแลตมินต์','มินต์ช็อคโกแลต']:
            ret_wtkn[i] = 'ช็อกโกแลตมินต์'
        elif each  in ['ไขมันต่ำ','low fat','พร่องมันเนย']:
            ret_wtkn[i] = 'ไขมันต่ำ'
        elif each  in ['ไขมัน 0','0 fat','ขาดมันเนย']:
            ret_wtkn[i] = 'ไขมัน 0%'
        elif each  in ['high protein','hi protein','ไฮโปรตีน']:
            ret_wtkn[i] = 'ไฮโปรตีน'
        elif each  in ['free lactose','lactose free','ฟรีแลคโตส','แลคโตสฟรี']:
            ret_wtkn[i] = 'นมฟรีแลคโตส'
        elif each  in ['พาสเจอไรซ์','พาสเจอร์ไรซ์','พาสเจอร์ไรส์','พาซเจอไรซ์','พาสเจอร์ไลท์']:
            ret_wtkn[i] = 'พาสเจอร์ไรส์'
        elif each  in ['เมจิโกลด์ แม็กซ์','เมจิโกลด์','เมจิโกล์ดแม็กซ์','gold','gold max']:
            ret_wtkn[i] = 'เมจิโกลด์'
        elif each  in ['ฝาน้ำเงิน','รสจืด']:
            ret_wtkn[i] = 'รสจืด'
        elif each  in ['ท็อปส์','ทอปส์','ท้อปส์','ท๊อปส์','tops']:
            ret_wtkn[i] = 'tops'
        elif each  in ['แมคโคร','แม็คโคร','makro']:
            ret_wtkn[i] = 'makro'
        elif each  in ['โลตัส','lotus']:
            ret_wtkn[i] = 'lotus'
        elif each  in ['บิ๊กซี','bigc','Big C']:
            ret_wtkn[i] = 'bigc'
        elif each  in ['ท้องเสีย','ขี้แตก']:
            ret_wtkn[i] = 'ท้องเสีย'
        elif each  in ['ลูกสุนัข','ลูกหมา']:
            ret_wtkn[i] = 'ลูกสุนัข'
        elif each in ['ดาร์คช็อกโกแลต','ดาร์กช็อก','ดาร์กช็อกโกแลต']:
            ret_wtkn[i] = 'ดาร์คช็อกโกแลต'
        elif each in ['bedtime','bed time','เบดไทม์']:
            ret_wtkn[i] = 'เบดไทม์'
        elif each in ['เมจิ บัลแกเรีย','เมจิบัลแกเรีย']:
            ret_wtkn[i] = 'เมจิบัลแกเรีย'
        elif each in ['โยเกิร์ต','โยเกิต','โยเกิรต','โยเกิร์ตพร้อมดื่ม']:
            ret_wtkn[i] = 'โยเกิร์ต'
        elif each in ['7-Eleven','เซเว่น อีเลฟเว่น','เซเว่นอีเลฟเว่น','เซเว่น','เซเวน','7/11','7-11']:
            ret_wtkn[i] = '7-Eleven'
        elif each in ['บีทาเก้น','บีทาเกน']:
            ret_wtkn[i] = 'บีทาเก้น'
        elif each in ['โปร','โปรโมชั่น']:
            ret_wtkn[i] = 'โปรโมชั่น'
    return ret_wtkn

In [ ]:
def find_brands_product(x,catType):
    ret_wtkn = []
    brands = ['ดัชมิลล์','เมจิ','โฟร์โมสต์','โชคชัย','แดรี่โฮม','เอ็มมิลค์','แมคโนเลีย','ไทยเดนมาร์ค','หนองโพ','คาเนชั่น','บีทาเก้น','จิตรลดา']
    products = ['สตรอว์เบอร์รี','ช็อกโกแลต','รสกาแฟ','รสหวาน','รสจืด','ไขมันต่ำ','ไขมัน 0%','ไฮโปรตีน','อัลมอนด์'
                ,'รสกล้วย','grass fed','นมฟรีแลคโตส','เมจิโกลด์','นมฮอกไกโด','เบดไทม์','ดาร์คช็อกโกแลต','ไฮแคลเซียม'
                ,'คาราเมล','มอลต์','เมล่อน','ชาเขียวมัจฉะ','บัลแกเรีย','รสธรรมชาติ','รสกลมกล่อม','ซากุระ']
    milk_kind=['นม','นมข้น','นมจืด','นมสด','กินนม','ดื่มนม','ขวดนม','นมวัว','นมกล่อง','ผลิตภันท์นม','น้ำนมโค'
                     ,'โยเกิร์ต','นมเปรี้ยว','uht','นมถั่วเหลือง','นมผง','พาสเจอร์ไรส์']
    
    choiceList = []
    if catType == 'b':
        choiceList = brands
    elif catType == 'm':
        choiceList = milk_kind
    elif catType == 'p':
        choiceList = products
        
    for i,each in enumerate(x,0):
        if each in choiceList:
            ret_wtkn.append(each)
    return ret_wtkn

#### Copy ชุดคำจาก File มา Process ก่อนลง MongoDB

In [ ]:
df_process_comment = df_Corpus.reset_index().copy()
df_process_thread = df_thread.reset_index().copy()

2.1) Tokenize comment

In [ ]:
df_process_comment['token_text'] = df_process_comment['text'].apply(lambda x: tokenize_custom(x,trie))

In [ ]:
df_process_comment.head()

2.2) Generalized keyword (Brand & product)

In [ ]:
df_process_comment['token_text'] = df_process_comment['token_text'].apply(lambda x: generalize_keyword(x))

In [ ]:
df_process_comment['cmt_mention_dairy'] = df_process_comment['token_text'].apply(lambda x: find_brands_product(x,'m'))
df_process_comment['cmt_mention_brand'] = df_process_comment['token_text'].apply(lambda x: find_brands_product(x,'b'))
df_process_comment['cmt_mention_product'] = df_process_comment['token_text'].apply(lambda x: find_brands_product(x,'p'))

2.3) Insert comments to DB

In [ ]:
col_comment.delete_many({})
process_data = create_InsertMany_comments(df_process_comment)
col_comment.insert_many(process_data)

2.4) Tokenize head of thread

In [ ]:
df_process_thread['token_headline'] = df_process_thread['headline'].apply(lambda x: tokenize_custom(x,trie))

2.5) Generalized keyword (Brand & product)

In [ ]:
df_process_thread['token_headline'] = df_process_thread['token_headline'].apply(lambda x: generalize_keyword(x))

2.6) Mark domain/entity mention

In [ ]:
df_process_thread['t_mention_dairy'] = df_process_thread['token_headline'].apply(lambda x: find_brands_product(x,'m'))
df_process_thread['t_mention_brand'] = df_process_thread['token_headline'].apply(lambda x: find_brands_product(x,'b'))
df_process_thread['t_mention_product'] = df_process_thread['token_headline'].apply(lambda x: find_brands_product(x,'p'))

In [ ]:
df_process_thread.head()

2.7) Insert thread to DB

In [ ]:
col_thread.delete_many({})
process_data = create_InsertMany_Thread(df_process_thread)
col_thread.insert_many(process_data)

In [ ]:
del df_thread,df_Corpus,df_process_comment,df_process_thread

### 3.Process หัวกระทู้ เพื่อทำ Topic Modeling

เคสหลุดเงื่อนไข แต่ต้องใช้
1. https://pantip.com/topic/30833944
2. https://pantip.com/topic/30105850
4. https://pantip.com/topic/35439062

In [ ]:
execption_urls = ['https://pantip.com/topic/30833944','https://pantip.com/topic/30105850','https://pantip.com/topic/35439062']

In [ ]:
cursor_thread = col_thread.find()
df_thr_process = pd.DataFrame(cursor_thread)

In [ ]:
cursor_comment = col_comment.find()
df_cmt_process = pd.DataFrame(cursor_comment)

In [ ]:
df_urls_milk = df_thr_process[(df_thr_process.t_mention_dairy.str.len() != 0) | (df_thr_process.URLs.isin(execption_urls))]
df_urls_milk['t_mention_dairy'] = df_urls_milk['t_mention_dairy'].apply(lambda x: repr(set(x)))
df_urls_milk['t_mention_brand'] = df_urls_milk['t_mention_brand'].apply(lambda x: repr(set(x)))
df_urls_milk['t_mention_product'] = df_urls_milk['t_mention_product'].apply(lambda x: repr(set(x)))

In [ ]:
df_c_urls_milk = df_cmt_process[(df_cmt_process.cmt_mention_dairy.str.len() != 0) | (df_cmt_process.URLs.isin(execption_urls))]
df_c_urls_milk['cmt_mention_dairy'] = df_c_urls_milk['cmt_mention_dairy'].apply(lambda x: repr(set(x)))
df_c_urls_milk['cmt_mention_brand'] = df_c_urls_milk['cmt_mention_brand'].apply(lambda x: repr(set(x)))
df_c_urls_milk['cmt_mention_product'] = df_c_urls_milk['cmt_mention_product'].apply(lambda x: repr(set(x)))

In [ ]:
print('จำนวนกระทู้ที่มีการพูดถึงนม:',df_urls_milk.shape[0])
print('%จำนวนกระทู้ที่มีการพูดถึงนมเทียบทั้งหมด:',round((df_urls_milk.shape[0]/df_thr_process.shape[0])*100,2))
print('จำนวน comment ที่มีการพูดถึงนม:',df_c_urls_milk.shape[0])
print('%จำนวน comment ที่มีการพูดถึงนมเทียบทั้งหมด:',round((df_c_urls_milk.shape[0]/df_cmt_process.shape[0])*100,2))

* จำนวน comment ที่มีการพูดถึงนม ไม่ได้ impute มาจากหัวกระทู้ ถ้า impute จะมีเยอะกว่านี้อีก

### EDA#2

การระบุ domain ของนม บนหัวกระทู้

การระบุชื่อ Product บนหัวกระทู้ (Headline)

การระบุชื่อ Product ใน comment ทั้งใต้หัวกระทู้ ที่มี domain ของนม และส่วน comment<br>
อย่างไรก็ตามยังไม่ได้ impute domain จากหัวกระทู้เข้าไป

การระบุชื่อ brand ใน comment

### 4.สรุป Term & reduced_keyword

In [ ]:
execption_urls = ['https://pantip.com/topic/30833944','https://pantip.com/topic/30105850','https://pantip.com/topic/35439062']

In [ ]:
attrributes = ['การบูด','โปร','โปรโมชั่น','แถม','บูด','เป็นผลดี','หา','หมดอายุ', 'กลิ่น','หอม','กลิ่นหอม', 'แนะนำ', 'กิน', 'shelf life','แม่','คุณแม่'
 ,'ลูก','เด็ก','เลี้ยงลูก','เลี้ยงเด็ก','พัฒนาการ','คุณแม่มือใหม่','สั่ง', 'ค่าใช้จ่าย', 'เพิ่มขึ้น', 'สินค้า', 'ชอบ','นิยม', 'สี', 'ซื้อไม่ได้'
 , 'ทิ้ง', 'จืด','หวาน' , 'fat', 'นิยม','ราคาสูง','กาแฟ','ชงกาแฟ','หาย','ต้ม','อุ่น','ร้อน', 'พื้นฐาน', 'ขายไม่ดี','สูง','ส่วนสูง'
 , 'กาแฟสด', 'จ่าย', 'ขาย', 'รส','รสชาติ', 'กลมกล่อม','ไขมัน','พนักงาน','ไมโครเวฟ','ฟาร์ม'
 ,'ส่วนผสม','ผสม','เบาหวาน','รสพื้นฐาน','โปรตีน','เวย์','เวท','วิ่ง','protein','whey','ขับถ่าย','อึ','dha','น้ำตาล','เบาหวาน'
 ,'ฝา', 'นมสด','ถ้วย', 'ร้าน', 'ดื่ม', 'เซเว่น', 'ขวด', 'นึกถึง', 'เสียใจ', 'แตก', 'ซื้อ','ขาย','มีลูก','เทรนเนอร์'
 ,'ขายไม่ดี','แพคคู่','ค่าจัดส่ง','shelf life','พนักงานขายนม','แพ้นม','แพ้นมวัว','เล่นเวท','ฝาน้ำเงิน','ฝาสีเขียว'
 ,'นมอุ่น','ชานม','กินนม','ดื่มนม','ท้องเสีย','ลูกสุนัข','สุนัข','หมา','แมว','คายทิ้ง','เจมส์จิ','แป้ง','ลดราคา'
 ,'ซื้อประจำ','ซื้อไม่ได้','คาปูชิโน่','อเมริกาโน่','ร้านนม','whey formula','ผิดสังเกต','เสียความรู้สึก','ชี้แจง','บำรุง','น้ำผึ้ง'
 ,'อาหารเสริม','มีประโยชน์','วิตามิน','นมผง','แคลเซี่ยม','ท้อง','แพะ','อร่อย','โภชนาการ','โรงเรียน','พ่อแม่','ครู','ยูเอชที'
 ,'ผู้บริโภค','ขโมย','ไอโอดีน','นมข้นหวาน','เนย','ตรวจสอบ','แกลลอน','มันดี','นมวัว','อาเจียน','เวฟ','ไมโครเวฟ','สารอาหาร'
 ,'ขนส่ง','ถูก','แพง','เค้ก','ทิ้ง','วันหมดอายุ','โอเมก้า','กล่อง','พลังงาน','โภชนาการ','ขนมปัง','ของแถม','ราคาสูง','น้ำนมโค'
 ,'บรรจุภัณฑ์','ความแข็งแรง','แข็งแรง','พรีเซ็นเตอร์','ญี่ปุ่น','น้ำผลไม้','ออกกำลังกาย','ประหยัด','วิปปิ้ง','เชื้อจุลินทรีย์'
 ,'แลคโตส']

stores = ['tops','makro','lotus','bigc','7-Eleven']

products = ['สตรอว์เบอร์รี','ช็อกโกแลต','รสกาแฟ','รสหวาน','รสจืด','ไขมันต่ำ','ไขมัน 0%','ไฮโปรตีน','อัลมอนด์'
                ,'รสกล้วย','grass fed','นมฟรีแลคโตส','เมจิโกลด์','นมฮอกไกโด','เบดไทม์','ดาร์คช็อกโกแลต','ไฮแคลเซียม'
                ,'คาราเมล','มอลต์','เมล่อน','ชาเขียวมัจฉะ','บัลแกเรีย','รสธรรมชาติ','รสกลมกล่อม','ซากุระ']
milk_kind=['นม','นมข้น','นมจืด','นมสด','กินนม','ดื่มนม','ขวดนม','นมวัว','นมกล่อง','ผลิตภันท์นม','น้ำนมโค'
                     ,'โยเกิร์ต','นมเปรี้ยว','uht','นมถั่วเหลือง','นมผง','พาสเจอร์ไรส์']

#  EDA ใน Excel ได้ flavor 68 ตัว (ถ้าดึงยี่ห้ออื่นก็เพิ่มอีก)
avai_flavs = ['เมจิเมจิโกลด์','เมจิเมล่อน','เมจิไขมัน 0%','เมจิไขมันต่ำ','เมจิไฮโปรตีน','เมจิช็อกโกแลต','เมจิชาเขียวมัจฉะ','เมจิดาร์คช็อกโกแลต'
 ,'เมจินมฟรีแลคโตส','เมจิบัลแกเรีย','เมจิมอลต์','เมจิรสกลมกล่อม','เมจิรสกล้วย','เมจิรกาแฟ','เมจิรสจืด','เมจิรสธรรมชาติ','เมจิรสหวาน'
 ,'เมจิสตรอว์เบอร์รี','เมจิอัลมอนด์','เอ็มมิลค์นมฟรีแลคโตส','เอ็มมิลค์รสจืด','แดรี่โฮมgrass fed','แดรี่โฮมเบดไทม์','แดรี่โฮมช็อกโกแลต'
 ,'แดรี่โฮมรสกล้วย','แดโฮมรสจืด','แดรี่โฮมรสหวาน','แดรี่โฮมสตรอว์เบอร์รี','แมคโนเลียไขมันต่ำ','แมคโนเลียช็อกโกแลต','แมคโนเลียรสจืด'
 ,'โชคชัยไขมันต่ำ','โชคชัยช็อกโกแลต','โชคชัยรสกาแฟ','โชคชัยรสจืด','โชคชัยสตรอว์เบอร์รี','ฟร์โมสต์ไขมัน 0%','โฟร์โมสต์ไขมันต่ำ'
 ,'โฟร์โมสต์คาราเมล','โฟร์โมสต์ช็อกโกแลต','โฟร์โมสต์รสกาแฟ','โฟร์โมสต์รสจืด','โฟร์โมสต์สตรอว์เบอร์รี','ไทยเดนมาร์คช็อกโกแลต'
 ,'ไทยเดนมาร์ครสกาแฟ','ไทยเดนมาร์ครสจืด','ไทยเดนมาร์ครสหวาน','ไทยเดนมาร์คสตรอว์เบอร์รี','คาเนชั่นรสจืด','จิตรลดาช็อกโกแลต'
 ,'จิตรลดารสจืด','จิตรลดารสหวาน','จิตรลดาสตรอว์เบอร์รี','ดัชมิลล์ไขมัน 0%','ดัชมิลล์ไขมันต่ำ','ดัชมิลล์ไฮโปรตีน','ดัชลล์อกกแลต'
 ,'ดัชมิลล์มอลต์','ดัชมิลล์รสกาแฟ','ดัชมิลล์รสจืด','ดัชมิลล์สตรอว์เบอร์รี','หนองโพไขมัน 0%','หนองโพไขมันต่ำ','หนองโพช็อกโกแลต'
 ,'หนองโพรสกาแฟ','หนองโพรสจืด','หนองโพรสหวาน','หนองโพสตรอว์เบอร์รี']

reduceCol = attrributes + stores 
reduceCol_all = attrributes + stores + milk_kind + products

In [ ]:
def reduced_keyword(wtkn,redCol):
    del_list = []
    ret_wtkn = wtkn
    for each in ret_wtkn:
        if each not in redCol:
            del_list.append(each)
    ret_wtkn = [x for x in ret_wtkn if x not in del_list]
    return ret_wtkn

In [ ]:
def freq_brand(x):
    #e = eval(x)
    e = x
    e.sort()
    f = [(k,len(list(g))) for k, g in groupby(e)]
    return f

In [ ]:
def concat_flavor(x,y):
    listflav = []
    for i in x:
        for j in y:
            listflav.append(i+j)
    return listflav

### 5.สร้าง (Reduce) Bag of word ด้วย dictionary.doc2bow จัดลง dataframe

In [ ]:
cursor_thread = col_thread.find()
df_thr_process = pd.DataFrame(cursor_thread)

In [ ]:
cursor_comment = col_comment.find()
df_cmt_process = pd.DataFrame(cursor_comment)

In [ ]:
# คัด headline ที่พูดถึง domain นม
df_urls_milk = df_thr_process[(df_thr_process.t_mention_dairy.str.len() != 0) | (df_thr_process.URLs.isin(execption_urls))]

In [ ]:
# เมื่อ join กันแล้ว คอมเม้นทุกคอมเม้นจะอยู่ใน domain นมทั้งหมด
df_join_url = pd.merge(df_urls_milk,df_cmt_process,how='inner',on='URLs')

In [ ]:
df_join_url.drop(columns=['_id_x','_id_y','headline_y'],inplace=True)
df_join_url.rename(columns={"headline_x":"headline"},inplace=True)

In [ ]:
# ทดลอง QAP mathod case2,4
df_join_url['token_text_reduce'] = df_join_url['token_text'].apply(lambda x: reduced_keyword(x, reduceCol))

# ทดลอง QAP mathod  case1,5
#df_join_url['token_text_reduce'] = df_join_url['token_text'].apply(lambda x: reduced_keyword(x, reduceCol_all))

In [ ]:
#df_join_url.set_index('commentId',inplace=True)
df_join_url.head(2)

In [ ]:
dictionary = gensim.corpora.Dictionary(df_join_url['token_text_reduce'])

In [ ]:
df_join_url['reduce_bow'] = df_join_url["token_text_reduce"].map(dictionary.doc2bow)
df_join_url['reduce_bow_txt'] = df_join_url["reduce_bow"].apply(lambda x:[(dictionary[id_], frequence) for id_, frequence in x])

In [ ]:
df_join_url.drop(columns=['Retrived_date','reduce_bow'],inplace=True)

In [ ]:
df_join_url.insert(3,'mention_domain',value=None)
df_join_url.insert(4,'mention_product',value=None)
df_join_url.insert(5,'mention_brand',value=None)

Impute brand & product เข้า comment และสรุปก่อนเข้า BoW

In [ ]:
for i,e in enumerate(df_join_url.index,0):
    if (len(df_join_url.t_mention_dairy.iloc[i]) > 0) & (len(df_join_url.cmt_mention_dairy.iloc[i]) > 0):         # 1.
        df_join_url['mention_domain'].iloc[i] = df_join_url.cmt_mention_dairy.iloc[i]
    elif  (len(df_join_url.t_mention_dairy.iloc[i]) > 0) & (len(df_join_url.cmt_mention_dairy.iloc[i]) == 0):  # 2.
        df_join_url['mention_domain'].iloc[i] = df_join_url.t_mention_dairy.iloc[i]
    elif  (len(df_join_url.t_mention_dairy.iloc[i]) == 0) & (len(df_join_url.cmt_mention_dairy.iloc[i]) > 0):  # 3.
        df_join_url['mention_domain'].iloc[i] = df_join_url.cmt_mention_dairy.iloc[i]
    else:
        df_join_url['mention_domain'].iloc[i] = None

In [ ]:
for i,e in enumerate(df_join_url.index,0):
    if (len(df_join_url.t_mention_brand.iloc[i]) > 0) & (len(df_join_url.cmt_mention_brand.iloc[i]) > 0):         # 1.
        df_join_url['mention_brand'].iloc[i] = df_join_url.cmt_mention_brand.iloc[i]
    elif  (len(df_join_url.t_mention_brand.iloc[i]) > 0) & (len(df_join_url.cmt_mention_brand.iloc[i]) == 0):  # 2.
        df_join_url['mention_brand'].iloc[i] = df_join_url.t_mention_brand.iloc[i]
    elif  (len(df_join_url.t_mention_brand.iloc[i]) == 0) & (len(df_join_url.cmt_mention_brand.iloc[i]) > 0):  # 3.
        df_join_url['mention_brand'].iloc[i] = df_join_url.cmt_mention_brand.iloc[i]
    else:
        df_join_url['mention_brand'].iloc[i] = None

In [ ]:
for i,e in enumerate(df_join_url.index,0):
    if (len(df_join_url.t_mention_product.iloc[i]) > 0) & (len(df_join_url.cmt_mention_product.iloc[i]) > 0):         # 1.
        df_join_url['mention_product'].iloc[i] = df_join_url.cmt_mention_product.iloc[i]
    elif  (len(df_join_url.t_mention_product.iloc[i]) > 0) & (len(df_join_url.cmt_mention_product.iloc[i]) == 0):  # 2.
        df_join_url['mention_product'].iloc[i] = df_join_url.t_mention_product.iloc[i]
    elif  (len(df_join_url.t_mention_product.iloc[i]) == 0) & (len(df_join_url.cmt_mention_product.iloc[i]) > 0):  # 3.
        df_join_url['mention_product'].iloc[i] = df_join_url.cmt_mention_product.iloc[i]
    else:
        df_join_url['mention_product'].iloc[i] = None

In [ ]:
df_join_url.drop(columns=['t_mention_dairy','t_mention_product','t_mention_brand'
                          ,'cmt_mention_dairy','cmt_mention_product','cmt_mention_brand'],inplace=True)

concat Brand + Product มาต่อกันเพื่อระบุตราและยี่ห้อ เพื่อลงระดับ flavor

Create cooccurence matrix

In [ ]:
# comment ออกเมื่อไม่ concat brand+flavor
df_final = df_join_url.copy()
df_final.set_index('commentId',inplace=True)
df_final['mention_product'] = df_final['mention_product'].apply(lambda x: x if x is None else freq_brand(x))
df_final['mention_brand'] = df_final['mention_brand'].apply(lambda x: x if x is None else freq_brand(x))
df_final['mention_domain'] = df_final['mention_domain'].apply(lambda x: x if x is None else freq_brand(x))

In [ ]:
df_brand_cooc = pd.concat([pd.DataFrame(s,columns=['w','c']).set_index('w') for s in df_final["mention_brand"]], axis=1, sort=False).fillna(0).T.set_index(df_final.index)
df_prd_cooc = pd.concat([pd.DataFrame(s,columns=['w','c']).set_index('w') for s in df_final["mention_product"]], axis=1, sort=False).fillna(0).T.set_index(df_final.index)
df_cooc = pd.concat([pd.DataFrame(s,columns=['w','c']).set_index('w') for s in df_final["reduce_bow_txt"]], axis=1, sort=False).fillna(0).T.set_index(df_final.index)

In [ ]:
df_brand_cooc.reset_index(inplace=True)
df_prd_cooc.reset_index(inplace=True)
df_cooc.reset_index(inplace=True)

In [ ]:
# ทดลอง  QAP mathod case2,4
#join_df = pd.merge(df_brand_cooc, df_prd_cooc, on='commentId', how='inner', suffixes=(False, False))
#join_df = pd.merge(join_df, df_cooc, on='commentId', how='inner')

# ทดลอง  QAP mathod case3
#join_df = pd.merge(df_prd_cooc, df_cooc, on='commentId', how='inner', suffixes=(False, False))

# ทดลอง  QAP mathod case1,5
join_df = pd.merge(df_brand_cooc, df_cooc, on='commentId', how='inner', suffixes=(False, False))


join_df.set_index('commentId',inplace=True)

In [ ]:
join_df.head()

Export ข้อมูล เพื่อนำไปสุ่มแถวทำ QAP

In [ ]:
join_df.to_csv(root_output_path+"/cooc_bow_all.csv",index=True)

## Create frequency co-occurrence matrix จาก Bag of word<br>
* ใช้สำหรับทำ MDS และโยนเข้า R (ทำ normalized บน R) ในลักษณะ Term-Term Cross tabulation<br> 
(คนละอย่างกับ TF-IDF cooc matrix ที่เป็น Word-word ซึ่งต้องรัน bigram เป็นราย document)

In [ ]:
item_item_matrix = pd.DataFrame(index=join_df.columns,columns=join_df.columns).fillna(0)

In [ ]:
def cnt_Grpby(df,i,j):
    PAIB = pd.DataFrame(df.groupby(df.iloc[:,i].name)[df.iloc[:,j].name].value_counts())
    PAIB.index.names = ['A','B']
    PAIB.columns = ['freq']
    PAIB.reset_index(inplace=True)
    _PAIB = PAIB[(PAIB.A>0)&(PAIB.B>0)]
    return _PAIB.freq.sum()

In [ ]:
# I like machine learning love NLP
#cnt_Grpby(pd.DataFrame([[1,1,1,1,0,0],[1,0,0,0,1,1]]),0,2)

In [ ]:
# Co-occurrence
for i in range(0,len(item_item_matrix.columns)):
    for j in range(0,len(item_item_matrix.columns)):
        if i != j:
            item_item_matrix.iloc[i,j] = cnt_Grpby(join_df,i,j)
        else:
            item_item_matrix.iloc[i,j] = 0

In [ ]:
item_item_matrix.head()

In [ ]:
item_item_matrix.to_excel(comment_cooc_freq_file, index=True)

### Create co-occurrence matrix with Lift normalization

In [ ]:
item_item_matrix = pd.DataFrame(index=join_df.columns,columns=join_df.columns).fillna(0)

In [ ]:
def a_in_b(df,i,j):
    PAIB = pd.DataFrame(df.groupby(df.iloc[:,i].name)[df.iloc[:,j].name].value_counts())
    PAIB.index.names = ['A','B']
    PAIB.columns = ['freq']
    PAIB.reset_index(inplace=True)
    PAIB_ = PAIB[(PAIB.A>0)&(PAIB.B>0)]
    return PAIB_.freq.sum()/PAIB.freq.sum()

In [ ]:
for i in range(0,len(item_item_matrix.columns)) :
    for j in range(0,len(item_item_matrix.columns)) :
        A = join_df.iloc[:,i]
        B = join_df.iloc[:,j]
        #PA = A[A!=0].count()/A.shape[0]
        #PB = B[B!=0].count()/B.shape[0]
        #PB_ = B[B==0].count()/B.shape[0]
        #PAB = PB*a_in_b(join_df,i,j)
        #item_item_matrix.iloc[i,j] = PAB/(PA*PB)
        item_item_matrix.iloc[i,j] = lift_score(A,B)

In [ ]:
item_item_matrix.replace([np.inf, -np.inf], np.nan,inplace=True)
item_item_matrix.fillna(0,inplace=True)

In [ ]:
item_item_matrix.head()

In [ ]:
item_item_matrix.to_excel(comment_cooc_lift_file, index=True)

### Create co-occurrence matrix with Jaccard Distance<br>
* สำหรับทำ Robusness check

In [ ]:
item_item_matrix = pd.DataFrame(index=join_df.columns,columns=join_df.columns).fillna(0)

In [ ]:
# Co-occurrence
for i in range(0,len(item_item_matrix.columns)):
    for j in range(0,len(item_item_matrix.columns)):
        item_item_matrix.iloc[i,j] = jaccard_score(join_df.iloc[i],join_df.iloc[j], average='weighted')
item_item_matrix.fillna(0,inplace=True)

In [ ]:
item_item_matrix.head()

In [ ]:
item_item_matrix.to_excel(comment_cooc_jacc_file, index=True)

### Create pearson correlation matrix<br>
* สำหรับทำ Robusness check

In [ ]:
item_item_matrix = pd.DataFrame(index=join_df.columns,columns=join_df.columns).fillna(0)

In [ ]:
# Co-occurrence
for i in range(0,len(item_item_matrix.columns)):
    for j in range(0,len(item_item_matrix.columns)):
        item_item_matrix.iloc[i,j] = scipy.stats.pearsonr(join_df.iloc[i],join_df.iloc[j])[0]
item_item_matrix.fillna(0,inplace=True)

In [ ]:
item_item_matrix.head()

In [ ]:
item_item_matrix.to_excel(comment_cooc_corr_file, index=True)

### Create cosine similarity correlation matrix<br>
* สำหรับทำ Robusness check

In [ ]:
item_item_matrix = pd.DataFrame(index=join_df.columns,columns=join_df.columns).fillna(0)

In [ ]:
# Co-occurrence
for i in range(0,len(item_item_matrix.columns)):
    for j in range(0,len(item_item_matrix.columns)):
        item_item_matrix.iloc[i,j] = 1 - cosine(join_df.iloc[i],join_df.iloc[j])
item_item_matrix.fillna(0,inplace=True)

In [ ]:
item_item_matrix.head()

In [ ]:
item_item_matrix.to_excel(comment_cooc_cosine_file, index=True)

### สุ่มตัดจำนวน Row-Col เพื่อนำไปทำ QAP Test

สุ่ม Document 1/16, 1/8, 1/4, 1/2 เอามาทำ Cooc-matrix แล้วไล่ทำ QAP เทียบ 9000 doc กับ 4 ชุด dataset

In [ ]:
np.seterr(all='ignore')
df_cooc_frq = pd.read_csv(root_output_path+"/cooc_bow_all.csv",index_col=0)

In [ ]:
df_rand1 = df_cooc_frq.sample(frac=(1/16))
df_rand2 = df_cooc_frq.sample(frac=(1/8))
df_rand3 = df_cooc_frq.sample(frac=(1/4))
df_rand4 = df_cooc_frq.sample(frac=(1/2))

In [ ]:
print(df_rand1.shape[0],df_rand2.shape[0],df_rand3.shape[0],df_rand4.shape[0])

In [ ]:
def create_RandomcoccMat_Lift(df):
    item_item_matrix = pd.DataFrame(index=df.columns,columns=df.columns).fillna(0)
    for i in range(0,len(item_item_matrix.columns)) :
        for j in range(0,len(item_item_matrix.columns)) :
            A = df.iloc[:,i]
            B = df.iloc[:,j]
            item_item_matrix.iloc[i,j] = lift_score(A,B)
    item_item_matrix.replace([np.inf, -np.inf], np.nan,inplace=True)
    item_item_matrix.fillna(0,inplace=True)
    return item_item_matrix

In [ ]:
item_item_matrix = create_RandomcoccMat_Lift(df_rand1)
item_item_matrix.to_excel('lift1.xlsx', index=True)

In [ ]:
item_item_matrix = create_RandomcoccMat_Lift(df_rand2)
item_item_matrix.to_excel('lift2.xlsx', index=True)

In [ ]:
item_item_matrix = create_RandomcoccMat_Lift(df_rand3)
item_item_matrix.to_excel('lift3.xlsx', index=True)

In [ ]:
item_item_matrix = create_RandomcoccMat_Lift(df_rand4)
item_item_matrix.to_excel('lift4.xlsx', index=True)